In [3]:
from transformers import CamembertForSequenceClassification, CamembertTokenizerFast, CamembertConfig
import torch 
import numpy as np 


In [5]:
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = CamembertForSequenceClassification.from_pretrained("./results/camembert/")


In [22]:
text = "Innovation produits & services"
ipt = tokenizer.encode_plus(text, None, pad_to_max_length=True, max_length=512, return_token_type_ids=True, return_tensors="pt")

C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\transformers\tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [23]:
res = model(**ipt)
logits = res.logits

In [24]:
from torch import nn 

preds = nn.Sigmoid()(logits)

In [25]:
preds >= 0.2

tensor([[False, False,  True, False, False, False, False, False, False, False,
         False, False]])

In [26]:
LABELS = [
    "Evolution des procédés industriels",
    "Secteur Infrastructure & Déchéterie",
    "Innovation produits & services",
    "Gestion des bâtiments",
    "Production & distribution d'énergie",
    "Gestion des déchets",
    "Secteur Ville durable",
    "Secteur Agriculture & Zones rurales",
    "Mobilité des employés",
    "Secteur Eau & écosystèmes",
    "Ressources humaines",
    "Secteur Bois"
    ]

In [32]:
LABELS[np.array(preds.detach().cpu().numpy()>=0.5, dtype=int)]

TypeError: only integer scalar arrays can be converted to a scalar index

In [42]:
def predict(text, model, tokenizer):
    encoded_inputs = tokenizer.encode_plus(text, None, pad_to_max_length=True, max_length=512, return_token_type_ids=True, return_tensors="pt")
    outputs = model(**encoded_inputs)
    probabilities = nn.Sigmoid()(outputs.logits)
    predictions = list(np.array(preds.detach().cpu().numpy()>=0.5, dtype=int)[0])
    results = []
    for i, x in enumerate(predictions):
        if x == 1:
            results.append(LABELS[i])
    return results

In [43]:
predict("Protection des chiens", model, tokenizer)

['Innovation produits & services']